# Semanal Latinoamerica y USA importaciones semana 44,  26 octubre a 2 de noviembre

In [19]:
import pandas as pd
master=pd.read_csv(r"C:\Users\artuv\OneDrive\Bases_Chile_y_UvasPeru\Master.csv")
current_week=44

In [20]:
latam=master[(master['Region']=='LATIN AMERICA')& (master['ETD Week'].str.contains('2025'))]
latam=latam.rename(columns={'Dep_Week': 'Departure Week'}) 

In [21]:
line_to_run= '''RUN THIS LINE  in the terminal #python -m nbconvert --to html --no-input "Weekly.ipynb" --log-level CRITICAL
#python -m nbconvert --to webpdf --no-input "Weekly.ipynb" --TagRemovePreprocessor.enabled=True --PDFExporter.latex_command="['pdflatex', '{filename}', '--landscape']" '''
import warnings
warnings.filterwarnings('ignore')

### Polar Fruit Exports most recent

In [22]:
polar=master[((master['Exporter']=='POLAR FRUIT INT')| (master['Exporter']=='POLAR FRUIT INTERNATIONAL SAC'))& master['Year Country'].str.contains('2025')]
polar=polar.rename(columns={'Dep_Week': 'Departure Week','Year Country':'Exporting Country'})
polar=polar.sort_values(by=['Departure Week','Boxes']) # Sort polar before pivoting
polar_recent=polar[polar['Departure Week']==current_week]
polar_pivot=polar_recent.pivot_table(index=['Departure Week','Country','Importer'],
                                     columns=['Exporting Country','Specie','Variety'],
                                     values='Boxes',aggfunc='sum',margins=True).fillna(0)
country_rn={'2025-CHILE':'Chile','2025-PERU':'Peru'}
polar_pivot.rename(columns=country_rn,inplace=True)
polar_pivot=polar_pivot.style.format("{:,.0f}")
polar_pivot

,,Exporting Country,Peru,All
,,Specie,TABLE GRAPE,
,,Variety,RED GLOBE,
Departure Week,Country,Importer,,
44,DOMINICAN REPUBLIC,GRUPO RAMOS,"4,560","4,560"
All,,,"4,560","4,560"


## Latest Week in Imports from Chile and Peru  to the USA

In [23]:
usa25 = master[(master['Country'] == 'U.S.A.') & master['Year Country'].str.contains('2025')]
usa25 = usa25.rename(columns={'Dep_Week': 'Departure Week', 'Year Country': 'Exporting Country'})
usa_recent = usa25[usa25['Departure Week'] == current_week]

# --- Pivot ---
usa_pivot = (
    usa_recent.pivot_table(
        index=['Market', 'Importer', 'Exporter'],
        columns=['Exporting Country', 'Specie', 'Variety group'],
        values='Boxes',
        aggfunc='sum'
    )
    .fillna(0)
)

# --- Rename exporting countries ---
usa_rename = {'2025-CHILE': 'Chile', '2025-PERU': 'Peru'}
usa_pivot.rename(columns=usa_rename, inplace=True)

# --- Compute specie totals (NOT variety totals) ---
specie_totals = usa_pivot.groupby(level=[0, 1], axis=1).sum()

# Rebuild MultiIndex: add a 'TOTAL' under each specie
specie_totals.columns = pd.MultiIndex.from_tuples(
    [(c[0], c[1], 'TOTAL') for c in specie_totals.columns],
    names=usa_pivot.columns.names
)

# --- Merge and reorder so TOTAL comes right after each specie ---
combined = pd.concat([usa_pivot, specie_totals], axis=1)

# Reorder columns per specie block
new_order = []
for (country, specie), _ in combined.groupby(level=[0, 1], axis=1):
    subcols = [col for col in combined.columns if col[0] == country and col[1] == specie and col[2] != 'TOTAL']
    totalcol = [col for col in combined.columns if col[0] == country and col[1] == specie and col[2] == 'TOTAL']
    new_order.extend(subcols + totalcol)

usa_pivot = combined[new_order]

# --- Style ---
usa_pivot = (
    usa_pivot.style
    .format("{:,.0f}")
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ])
    .set_properties(**{'text-align': 'center'})
)
usa_pivot

In [24]:
countries = [
    'COLOMBIA', 'CHILE', 'COSTA RICA', 'ECUADOR', 
    'EL SALVADOR', 'HONDURAS', 'GUATEMALA','MEXICO', 
    'PANAMA', 'PERU', 'VENEZUELA'
]
country_pivots = {}
for country in countries:
    df_country = latam[latam['Country'] == country]
    df_country_new=df_country[df_country['Departure Week']==current_week]
    df_country_new=df_country_new.rename(columns={'Year Country':'Exporting Country'})
    pivot = (
        df_country_new.pivot_table(
            index=['Departure Week','Importer','Exporter',],
            columns=['Exporting Country', 'Specie', 'Variety group'],
            values='Boxes',
            aggfunc='sum',
            margins=True
        )
        .fillna(0)
    )
    yc_remap={'2025-CHILE':'Chile','2025-PERU':'Peru'}
    specie_totals = pivot.groupby(level=[0, 1], axis=1).sum()
    specie_totals.columns = pd.MultiIndex.from_tuples(
        [(c[0], c[1], 'TOTAL') for c in specie_totals.columns],
        names=pivot.columns.names
    )
    pivot = pd.concat([pivot, specie_totals], axis=1).sort_index(axis=1)
    pivot.rename(columns=yc_remap,inplace=True)
    pivot=pivot.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
    ]).set_properties(**{'text-align': 'center'})

    country_pivots[country] = pivot

In [25]:
countries = [
    'COLOMBIA', 'CHILE', 'COSTA RICA', 'ECUADOR', 
    'EL SALVADOR', 'HONDURAS', 'GUATEMALA', 'MEXICO', 
    'PANAMA', 'PERU', 'VENEZUELA'
]

country_pivots = {}
yc_remap = {'2025-CHILE': 'Chile', '2025-PERU': 'Peru'}

for country in countries:
    # Filter per country and current week
    df_country = latam[(latam['Country'] == country) & (latam['Departure Week'] == current_week)]
    if df_country.empty:
        continue

    df_country = df_country.rename(columns={'Year Country': 'Exporting Country'})

    # --- Base pivot ---
    pivot = (
        df_country.pivot_table(
            index=['Departure Week', 'Importer', 'Exporter'],
            columns=['Exporting Country', 'Specie', 'Variety group'],
            values='Boxes',
            aggfunc='sum'
        )
        .fillna(0)
    )

    # --- Rename exporting countries ---
    pivot.rename(columns=yc_remap, inplace=True)

    # --- Compute specie totals (NOT variety totals) ---
    specie_totals = pivot.groupby(level=[0, 1], axis=1).sum()
    specie_totals.columns = pd.MultiIndex.from_tuples(
        [(c[0], c[1], 'TOTAL') for c in specie_totals.columns],
        names=pivot.columns.names
    )

    # --- Merge & reorder so TOTAL appears right after each specie ---
    combined = pd.concat([pivot, specie_totals], axis=1)
    new_order = []
    for (country_label, specie), _ in combined.groupby(level=[0, 1], axis=1):
        subcols = [col for col in combined.columns 
                   if col[0] == country_label and col[1] == specie and col[2] != 'TOTAL']
        totalcol = [col for col in combined.columns 
                    if col[0] == country_label and col[1] == specie and col[2] == 'TOTAL']
        new_order.extend(subcols + totalcol)

    combined = combined[new_order]

    # --- Apply formatting & store ---
    styled = (
        combined.style
        .format("{:,.0f}")
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},
            {'selector': 'td', 'props': [('text-align', 'center')]}
        ])
        .set_properties(**{'text-align': 'center'})
    )

    country_pivots[country] = styled


   <p style="page-break-after:always;"></p>

## Latest Week in Imports from Chile and or  Peru for Latin America by Country

### Colombia

In [26]:
colombia_pivot = country_pivots['COLOMBIA']
colombia_pivot

### Chile

In [27]:
if country_pivots.keys().__contains__('CHILE'):
    chile_pivot = country_pivots['CHILE']
    chile_pivot
else:
    print (f'Nothing this week to Chile')

Nothing this week to Chile


<p style="page-break-after:always;"></p>

### Costa Rica

In [28]:
if country_pivots.keys().__contains__('COSTA RICA'):
    costa_rica_pivot = country_pivots['COSTA RICA']
else: 
    print (f'Nothing this week from Costa Rica')
costa_rica_pivot

### Ecuador

In [29]:
ecuador_pivot = country_pivots['ECUADOR']
ecuador_pivot

### El Salvador

In [30]:
if not country_pivots.keys().__contains__('EL SALVADOR'):
    print(f'Nothing this week to El Salvador')
else:
    el_sal_piv=country_pivots['EL SALVADOR']
    el_sal_piv

Nothing this week to El Salvador


### Guatemala

In [31]:
guatemala_pivot = country_pivots['GUATEMALA']
guatemala_pivot

<p style="page-break-after:always;"></p>

### Honduras

In [32]:
if not country_pivots.keys().__contains__('HONDURAS'):
    print(f'Nothing this week to Honduras')
else:
    honduras=country_pivots['HONDURAS']
    honduras

Nothing this week to Honduras


### Mexico

In [33]:
mx=country_pivots['MEXICO']
mx

   <p style="page-break-after:always;"></p>

### Panama

In [34]:
panama_pivot = country_pivots['PANAMA']
panama_pivot

### Peru

In [35]:
peru_pivot = country_pivots['PERU']
peru_pivot

<p style="page-break-after:always;"></p>

### Venezuela

In [36]:
venezuela_pivot = country_pivots['VENEZUELA']
venezuela_pivot